In [1]:
from api_wrapper.census_api import CensusAPI, CensusBoundaries, CensusDataAPI
from proj_paths.paths import Paths
import pandas as pd
import numpy as np

from src.data.load_sql import SqlExec

In [2]:
paths = Paths()

/home/mike/cavalier/projects/demographics-by-drivetime


In [3]:
summary_files = paths.data.census_description.summary_filetemplates.filepaths

In [4]:
def pad_list(lst, length=8, pad_value=None):
    
    lst_length = len(lst)
    elements_to_pad = length - lst_length
    
    return lst + elements_to_pad * [pad_value]

In [5]:
def parse_seq_excel(seq_excel_file):
    
    df = pd.read_excel(seq_excel_file)
    df = df.drop(df.columns[:6], axis=1)
    
    df = df.T
    df.columns = ['table_data']
    
    df['table_data'] = df.table_data.str.split('%')
    df = pd.DataFrame(df['table_data'].to_list(), index=df.index)
    
    return df

In [6]:
seq_dfs_lst = [parse_seq_excel(file) for file in summary_files if file.endswith('.xlsx') and 'seq' in file]

In [7]:
full_table_df = pd.concat(seq_dfs_lst, axis=0)

In [8]:
full_table_df.columns = ['overall_category'] + [f'subtitle_{n}' for n in range(8)]

In [9]:
full_table_df.index.name = 'table_id'

In [10]:
sql_exec = SqlExec()

In [11]:
sql_exec.create_tables()

In [12]:
full_table_df.to_sql('table_metadata', sql_exec.engine, if_exists='append', method="multi")

In [17]:
sql_exec.Base.metadata

MetaData(bind=None)